# Working with the lsstseries `Ensemble` object

When working with many lightcurves, the lsstseries `Ensemble` object serves as a singular interface for storing, filtering, and analyzing timeseries data. 
Let's consider an example set of lightcurves:

In [ ]:
from lsstseries.ensemble import Ensemble

ens = Ensemble()  # initialize an ensemble object

# Read in data from a parquet file
ens.from_parquet(
    "../../tests/lsstseries_tests/data/source/test_source.parquet",
    id_col="ps1_objid",
    time_col="midPointTai",
    flux_col="psFlux",
    err_col="psFluxErr",
    band_col="filterName",
)

We now have an `Ensemble` object, and have provided it with data from a parquet file. Within the call to `Ensemble.from_parquet`, we specified which columns of the input file mapped to timeseries quantities that the `Ensemble` needs to understand. It's important to link these arguments properly, as the `Ensemble` will use these columns when operations are requested on understood quantities. For example, if an lsstseries analysis function requires the time column, from this linking the `Ensemble` will automatically supply that function with the 'midPointTai' column.

## Column Mapping with the ColumnMapper

In the above example, we manually provide the column labels within the call to `Ensemble.from_parquet`. Alternatively, the `lsstseries.utils.ColumnMapper` class offers a means to assign the column mappings. Either manually as shown before, or even populated from a known mapping scheme. Since our dataset is from ZTF, we can utilize the known mapping for ZTF data.

In [ ]:
from lsstseries.utils import ColumnMapper

# columns assigned manually
manual_colmap = ColumnMapper().assign(
    id_col="ps1_objid", time_col="midPointTai", flux_col="psFlux", err_col="psFluxErr", band_col="filterName"
)

# columns assigned using known ZTF columns
ztf_map = ColumnMapper().use_known_map("ZTF")

# Pass the ColumnMapper along to read_parquet
ens.from_parquet("../../tests/lsstseries_tests/data/source/test_source.parquet", column_mapper=ztf_map)

## The Object and Source Frames
The `Ensemble` maintains two dataframes under the hood, the "object dataframe" and the "source dataframe". This borrows from the Rubin Observatories object-source convention, where object denotes a given astronomical object and source is the collection of measurements of that object. Essentially, the Object frame stores one-off information about objects, and the source frame stores the available time-domain data. As a result, Ensemble functions that operate on the underlying dataframes need to be pointed at either object or source. In most cases, the default is the object table as it's a more helpful interface for understanding the contents of the ensemble, especially when dealing with large volumes of data.

## Dask and "Lazy Evaluation"

Before going any further, the `Ensemble` is built on top of `Dask`, which brings with it a powerful framework for parallelization and scalability. However, there are some differences in how `Dask` code works that, if you're unfamiliar with it, is worth establishing right here at the get-go. The first is that `Dask` evaluates code "lazily". Meaning that many operations are not executed when the line of code is run, but instead are added to a scheduler to be executed when the result is actually needed. See below for an example:

In [ ]:
ens._source  # We have not actually loaded any data into memory

Here we are accessing the Dask dataframe underneath, and despite running a command to read in our data, we only see an empty dataframe with some high-level information available. To explicitly bring the data into memory, we must run a `compute()` command.

In [ ]:
ens.compute("source")  # Compute lets dask know we're ready to bring the data into memory

With this compute, we see above that we now have a populated dataframe (a Pandas dataframe in fact!). From this, many workflows in Dask and by extension lsstseries, will look like a series of lazily evaluated commands that are chained together and then executed with a .compute() call at the end of the workflow.

Alternatively we can use `ens.persist()` to execute the chained commands without loading the result into memory. This can speed up future `compute()` calls.

## Inspection and Filtering

The `Ensemble` contains an assortment of functions for inspecting and filtering your data.

### Inspection

These functions provide views into the contents of your `Ensemble` dataframe, especially important when dealing with large data volumes that cannot be brought into memory all at once. The first is `Ensemble.info` which provides information on the columns, data types, and memory usage of the dataframe.

In [ ]:
# Inspection

ens.info(verbose=True, memory_usage=True)  # Grabs high level information about the dataframes

`Ensemble.info` shows that we have 2000 rows with 54.7 KBs of used memory, and shows the columns we've brought in with their respective data types. If you'd like to actually bring a few rows into memory to inspect, `Ensemble.head` and `Ensemble.tail` provide access to the first n and last n rows respectively.

In [ ]:
ens.head("object", 5)  # Grabs the first 5 rows of the object table

In [ ]:
ens.tail("source", 5)  # Grabs the last 5 rows of the source table

Additionally, when you are working with a small enough dataset, `Ensemble.compute` can be used to bring the whole dataframe into memory (as shown previously). 

In [ ]:
ens.compute("source")

### Filtering

The `Ensemble` provides a general filtering function `query` that mirrors a Pandas or Dask `query` command. Specifically, the function takes a string that provides an expression indicating which rows to **keep**. As with other `Ensemble` functions, an optional `table` parameter allows you to filter on either the object or the source table.

For example, the following code filters the sources to only include rows with a flux value above 18.2. It uses `ens._flux_col` to retrieve the name of the column with that information.

In [ ]:
ens.query(f"{ens._flux_col} > 18.2", table="source")
ens.compute("source")

Alternatively, we could use a Dask dataseries of Booleans to indicate which rows to *keep*. We can often compute these series as the result of some operation on the underlying tables:

In [ ]:
keep_rows = ens._source[ens._err_col] < 0.05
keep_rows.compute()

We then pass that series to a `filter_from_series` function:

In [ ]:
ens.filter_from_series(keep_rows, table="source")
ens.compute("source")

Additionally, several more specific functions are available for common operations.

In [ ]:
# Cleaning nans
ens.dropna(threshold=1)  # threshold is the number of nans present in a row needed to drop the row

# Filtering on number of observations
ens.prune(threshold=50)  # threshold is the number of observations needed to retain the object

ens.info(verbose=True)

In the above operations, we remove any rows that have at least 1 NaN value present. And then filter such that only lightcurves which have at least 50 measurements are retained. We see then that `Ensemble.info` shows we've reduced the size of the dataset from 2000 rows to 1928 rows.

## Batch Analysis

The `Ensemble` provides a powerful batching interface, `Ensemble.batch`, with in-built parallelization (provided the input data is in multiple partitions). In addition, lsstseries has a suite of analysis functions on-hand for your use. Below, we show the application of `lsstseries.analysis.calc_stetson_J` on our dataset.

In [ ]:
# using lsstseries analysis functions
from lsstseries.analysis import calc_stetson_J

res = ens.batch(calc_stetson_J, compute=True)  # compute is set to true to execute immediately (non-lazily)
res

## Using a Custom Analysis Function
The analysis functions contained in lsstseries are meant to provide a collection of performant, on-hand routines for common timeseries use cases. However, lsstseries is also equipped to handle externally defined functions. Let's walk through a short example of defining a simple custom function and applying it through `Ensemble.batch`.

Here we define a simple function, that returns an average flux for each photometric band. It requires an array of fluxes, an array of band labels per measurement, and has a keyword argument for determining which averaging strategy to use (mean or median).

In [ ]:
import numpy as np


# Defining a simple function
def my_flux_average(flux_array, band_array, method="mean"):
    """Read in an array of fluxes, and return the average of the fluxes by band"""
    res = {}
    for band in np.unique(band_array):
        mask = [band_array == band]  # Create a band by band mask
        band_fluxes = flux_array[tuple(mask)]  # Mask the flux array
        if method == "mean":
            res[band] = np.mean(band_fluxes)
        elif method == "median":
            res[band] = np.median(band_fluxes)
    return res

With the function defined, we next supply it to `Ensemble.batch`. The column labels of the `Ensemble` columns we want to use as arguments must be provided, as well as any keyword arguments. In this case, we pass along `"psFlux"` and `"filterName"`, so that the `Ensemble` will map those columns to `flux_array` and `band_array` respectively. We also pass `method='mean'`, which will pass that kwarg along to `my_flux_average`.

In [ ]:
# Applying the function to the ensemble
res = ens.batch(my_flux_average, "psFlux", "filterName", compute=True, meta=None, method="median")
res

We see that we now have a `Pandas.series` of `my_average_flux` result by object_id (lightcurve). In many cases, this may not be the ideal output for your function. This output is controlled by the `Dask` `meta` parameter. For more information on this parameter, you can read the `Dask` [documentation](https://blog.dask.org/2022/08/09/understanding-meta-keyword-argument). You may pass the `meta` parameter through `Ensemble.batch`, as shown above.

In [ ]:
ens.client.close()  # Tear down the ensemble client